Tables
Experiments: To store information about each experiment.
Setups: To store setup-specific information, linked to experiments.
Channels: To store information about each channel within a setup.
HeaterSettings: To store heater settings for each setup.
Measurements: To store data from the CSV files.

In [ ]:
# ## Schema Design

# ### Experiments Table

# CREATE TABLE Experiments (
#     experiment_id INTEGER PRIMARY KEY,
#     experiment_name TEXT NOT NULL,
#     experiment_info TEXT,
#     experiment_time_ms INTEGER,
#     duration_s INTEGER,
#     repeat INTEGER,
#     heating_time_ms INTEGER
# );

# ### Setups Table
# CREATE TABLE Setups (
#     setup_id INTEGER PRIMARY KEY,
#     experiment_id INTEGER,
#     setup_number INTEGER,
#     FOREIGN KEY (experiment_id) REFERENCES Experiments (experiment_id)
# );

# ### Channels Table
# CREATE TABLE Channels (
#     channel_id INTEGER PRIMARY KEY,
#     setup_id INTEGER,
#     channel_number INTEGER,
#     channel_description TEXT,
#     FOREIGN KEY (setup_id) REFERENCES Setups (setup_id)
# );

# ### HeaterSettings Table
# CREATE TABLE HeaterSettings (
#     heater_setting_id INTEGER PRIMARY KEY,
#     setup_id INTEGER,
#     temperature INTEGER,
#     FOREIGN KEY (setup_id) REFERENCES Setups (setup_id)
# );

# ### Measurements Table
# CREATE TABLE Measurements (
#     measurement_id INTEGER PRIMARY KEY,
#     setup_id INTEGER,
#     channel_number INTEGER,
#     repeat_number INTEGER,
#     setting INTEGER,
#     timestamp INTEGER,
#     sensor_value REAL,
#     FOREIGN KEY (setup_id) REFERENCES Setups (setup_id)
# );


In [1]:
import sqlite3
import json
import os
import csv

# Connect to SQLite database
conn = sqlite3.connect('voc_lab.db')
cursor = conn.cursor()

# Create tables
cursor.execute('''
CREATE TABLE IF NOT EXISTS Experiments (
    experiment_id INTEGER PRIMARY KEY,
    experiment_name TEXT NOT NULL,
    experiment_info TEXT,
    experiment_time_ms INTEGER,
    duration_s INTEGER,
    repeat INTEGER,
    heating_time_ms INTEGER
);
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS Setups (
    setup_id INTEGER PRIMARY KEY,
    experiment_id INTEGER,
    setup_number INTEGER,
    FOREIGN KEY (experiment_id) REFERENCES Experiments (experiment_id)
);
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS Channels (
    channel_id INTEGER PRIMARY KEY,
    setup_id INTEGER,
    channel_number INTEGER,
    channel_description TEXT,
    FOREIGN KEY (setup_id) REFERENCES Setups (setup_id)
);
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS HeaterSettings (
    heater_setting_id INTEGER PRIMARY KEY,
    setup_id INTEGER,
    temperature INTEGER,
    FOREIGN KEY (setup_id) REFERENCES Setups (setup_id)
);
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS Measurements (
    measurement_id INTEGER PRIMARY KEY,
    setup_id INTEGER,
    channel_number INTEGER,
    repeat_number INTEGER,
    setting INTEGER,
    timestamp INTEGER,
    sensor_value REAL,
    FOREIGN KEY (setup_id) REFERENCES Setups (setup_id)
);
''')


In [2]:
# Check if the tables are created successfully
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

print("Tables in the database:")
for table in tables:
    print(table[0])

Tables in the database:
Experiments
Setups
Channels
HeaterSettings
Measurements


# Write local data to DB

In [13]:
import sqlite3
import os
import csv
import re

# Connect to SQLite database
conn = sqlite3.connect('voc_lab.db')
cursor = conn.cursor()

# Path to the folder containing the CSV files
csv_folder_path = r"D:\code\uom_explore\data\2024_07_23\exp_efficiency_test_async_1"  # Use raw string

# Function to extract setup, channel, and repeat numbers from filename
def extract_info_from_filename(filename):
    match = re.match(r'\d+_\d+_\d+s(\d+)c(\d+)r(\d+)\.csv', filename)
    if match:
        return int(match.group(1)), int(match.group(2)), int(match.group(3))
    return None, None, None

# Iterate through all files in the folder
for filename in os.listdir(csv_folder_path):
    if filename.endswith('.csv'):
        setup_number, channel_number, repeat_number = extract_info_from_filename(filename)
        if setup_number is not None:
            print(f"Processing file: {filename}")
            print(f"Extracted - Setup: {setup_number}, Channel: {channel_number}, Repeat: {repeat_number}")
            # Read CSV content
            with open(os.path.join(csv_folder_path, filename)) as csvfile:
                reader = csv.reader(csvfile)
                for row in reader:
                    if len(row) == 3:
                        setting, timestamp, sensor_value = row
                        # Insert data into Measurements table
                        cursor.execute('''
                        INSERT INTO Measurements (setup_id, channel_number, repeat_number, setting, timestamp, sensor_value)
                        VALUES ((SELECT setup_id FROM Setups WHERE setup_number = ?), ?, ?, ?, ?, ?)
                        ''', (setup_number, channel_number, repeat_number, setting, timestamp, sensor_value))
                    else:
                        print(f"Unexpected row format in file {filename}: {row}")
        else:
            print(f"Filename {filename} does not match the expected pattern")

# Commit changes and close the connection
conn.commit()

# Check if data was inserted
cursor.execute("SELECT COUNT(*) FROM Measurements;")
print(f"Number of records in Measurements table after insertion: {cursor.fetchone()[0]}")

conn.close()


Processing file: 02_17_16s1c0r0.csv
Extracted - Setup: 1, Channel: 0, Repeat: 0
Processing file: 02_17_51s1c1r0.csv
Extracted - Setup: 1, Channel: 1, Repeat: 0
Processing file: 02_18_26s1c2r0.csv
Extracted - Setup: 1, Channel: 2, Repeat: 0
Processing file: 02_19_01s1c3r0.csv
Extracted - Setup: 1, Channel: 3, Repeat: 0
Processing file: 02_19_36s1c4r0.csv
Extracted - Setup: 1, Channel: 4, Repeat: 0
Processing file: 02_20_11s1c0r1.csv
Extracted - Setup: 1, Channel: 0, Repeat: 1
Processing file: 02_20_47s1c1r1.csv
Extracted - Setup: 1, Channel: 1, Repeat: 1
Processing file: 02_21_22s1c2r1.csv
Extracted - Setup: 1, Channel: 2, Repeat: 1
Processing file: 02_21_57s1c3r1.csv
Extracted - Setup: 1, Channel: 3, Repeat: 1
Processing file: 02_22_32s1c4r1.csv
Extracted - Setup: 1, Channel: 4, Repeat: 1
Processing file: 02_23_07s1c0r2.csv
Extracted - Setup: 1, Channel: 0, Repeat: 2
Processing file: 02_23_42s1c1r2.csv
Extracted - Setup: 1, Channel: 1, Repeat: 2
Processing file: 02_24_17s1c2r2.csv
Extr

# Check results

In [14]:
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect('voc_lab.db')
cursor = conn.cursor()

# Function to check table structure
def check_table_structure(table_name):
    cursor.execute(f"PRAGMA table_info({table_name});")
    return cursor.fetchall()

# Function to count records in a table
def count_records(table_name):
    cursor.execute(f"SELECT COUNT(*) FROM {table_name};")
    return cursor.fetchone()[0]

# Function to query specific data from Measurements table
def query_measurements(limit=10):
    cursor.execute("SELECT * FROM Measurements LIMIT ?;", (limit,))
    return cursor.fetchall()

# Check the structure of each table
tables = ['Experiments', 'Setups', 'Channels', 'HeaterSettings', 'Measurements']
for table in tables:
    print(f"Structure of {table} table:")
    for column in check_table_structure(table):
        print(column)
    print()

# Count the records in each table
for table in tables:
    print(f"Number of records in {table} table: {count_records(table)}")
    print()

# Query and print some data from Measurements table
print("Sample data from Measurements table:")
for row in query_measurements():
    print(row)

# Close the connection
conn.close()


Structure of Experiments table:
(0, 'experiment_id', 'INTEGER', 0, None, 1)
(1, 'experiment_name', 'TEXT', 1, None, 0)
(2, 'experiment_info', 'TEXT', 0, None, 0)
(3, 'experiment_time_ms', 'INTEGER', 0, None, 0)
(4, 'duration_s', 'INTEGER', 0, None, 0)
(5, 'repeat', 'INTEGER', 0, None, 0)
(6, 'heating_time_ms', 'INTEGER', 0, None, 0)

Structure of Setups table:
(0, 'setup_id', 'INTEGER', 0, None, 1)
(1, 'experiment_id', 'INTEGER', 0, None, 0)
(2, 'setup_number', 'INTEGER', 0, None, 0)

Structure of Channels table:
(0, 'channel_id', 'INTEGER', 0, None, 1)
(1, 'setup_id', 'INTEGER', 0, None, 0)
(2, 'channel_number', 'INTEGER', 0, None, 0)
(3, 'channel_description', 'TEXT', 0, None, 0)

Structure of HeaterSettings table:
(0, 'heater_setting_id', 'INTEGER', 0, None, 1)
(1, 'setup_id', 'INTEGER', 0, None, 0)
(2, 'temperature', 'INTEGER', 0, None, 0)

Structure of Measurements table:
(0, 'measurement_id', 'INTEGER', 0, None, 1)
(1, 'setup_id', 'INTEGER', 0, None, 0)
(2, 'channel_number', 'INT